In [1]:
import pandas as pd
from datetime import datetime, timedelta
from database_manager import DatabaseManager
from helper import write_to_db

In [2]:
number = 1

df = pd.read_csv(datetime.strftime(datetime.now() - timedelta(number), '%d-%m-%Y') +'_coordinate_foods.csv')

In [3]:
df

,created_at,order_id,user_id,provider_id,restaurant_name,restaurant_lat,restaurant_lng,restaurant_payment_type,delivery_lat,delivery_lng,...,dist_pickedup_delivered,count_coordinates_accepted_pickedup,count_coordinates_pickedup_delivered,coordinates_accepted_pickedup,coordinates_pickedup_delivered,coord_pattern,lt100m,lt50m,gt10km,gt20km
0,2021-08-01 08:46:49,RH-2C5C0-6,7c55a9b2-76b4-4ae9-9bef-b3d2a5df62d7,867bea4a-6cb4-4f3b-b491-2ac64b212005,Sultan's Dine - Dhanmondi,23.742944,90.370972,1,23.749710,90.388382,...,319,301,66,"[(23.7413046, 90.383676), (23.7415272, 90.3838...","[(23.7500318, 90.3872629), (23.7500318, 90.387...",0,0,0,0,0
1,2021-08-01 10:54:26,MS-51C6D-22,f026fa9c-a816-45e2-a644-17fe591e29f4,bb9e697e-8fe4-47de-a17f-7c4540fc1306,7Dayz,22.367022,91.822365,1,22.360806,91.809189,...,5090,32,142,"[(22.3668919, 91.8227149), (22.3669084, 91.822...","[(22.3669458, 91.8223293), (22.3669771, 91.822...",0,0,0,0,0
2,2021-08-01 12:39:29,AP-84B87-6,a32e1727-6d46-498a-b704-7609e6a2eb78,b457d7cc-287f-4760-b2ec-713a3af006ef,Cafe Bleu,23.752709,90.381188,0,23.749073,90.382225,...,835,498,181,"[(23.7517949, 90.380879), (23.7518297, 90.3808...","[(23.7491637, 90.3820693), (23.7491788, 90.382...",0,0,0,0,0
3,2021-08-01 12:50:23,AS-AD41F-5,b3096a63-179c-4710-8482-2a6a1d077905,db80af2d-61ca-4955-a532-878207e2029d,Mr. Sub Khan,23.799166,90.383179,0,23.794886,90.384003,...,1446,159,233,"[(23.7991617, 90.3831567), (23.7990641, 90.383...","[(23.7991133, 90.3831667), (23.7991133, 90.383...",0,0,0,0,0
4,2021-08-01 15:01:15,MR-9CABC-4,71e2c01a-b09d-49b1-9fcf-888408affb5a,8f58f715-b2ff-4fa1-8bf8-97c36c05747c,Hazi Nanna Biryani 1 - Mirpur 10,23.810072,90.370426,0,23.799900,90.382423,...,1175,189,210,"[(23.8065953, 90.3666264), (23.8065957, 90.366...","[(23.8034716, 90.3761716), (23.8034717, 90.376...",0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,2021-08-01 11:54:44,MR-A068E-1,969fe312-d6a9-48c6-b902-3a8240bf3bcb,8a6bfd3c-21fb-4969-960d-6baad33251d7,The Mughal Empire,23.763041,90.361042,0,23.767878,90.367638,...,1416,98,137,"[(23.7631162, 90.3612515), (23.7631163, 90.361...","[(23.7631219, 90.3610575), (23.7631143, 90.361...",0,0,0,0,0
2024,2021-08-01 11:57:54,FM-E59A6-8,7d7f1bb1-323a-46d8-908e-732deb04bf41,88696949-87bb-4146-89e2-5f633b713771,Khana Ghat,22.356834,91.836577,0,22.337645,91.824768,...,1885,510,115,"[(22.3583426, 91.8371037), (22.3583633, 91.837...","[(22.3479853, 91.8328841), (22.3479853, 91.832...",0,0,0,0,0
2025,2021-08-01 12:31:58,MC-28D4C-7,c3bbe9df-af31-4a4b-af8c-8c6d137ced50,9e2549b0-4f1e-4fea-b751-7e2d43f46167,Chillox - Dhanmondi,23.739212,90.375348,0,23.739836,90.367004,...,1868,228,174,"[(23.7450481, 90.3765733), (23.7449712, 90.376...","[(23.7400693, 90.3747728), (23.7400813, 90.374...",0,0,0,0,0
2026,2021-08-01 11:22:31,RA-15E12-6,bc4d07cd-fc2d-4413-83f0-cf7bd8d8cbe7,d4308bc0-b3a0-4523-8423-3fa75e3212d5,The Sky Lounge & Restaurant,22.358722,91.825058,0,22.357716,91.817970,...,1520,135,119,"[(22.3661317, 91.82354), (22.3661317, 91.82354...","[(22.3589078, 91.8250694), (22.3589353, 91.825...",0,0,0,0,0


In [4]:
df['res_has_panel']

0       1
1       1
2       1
3       1
4       1
       ..
2023    1
2024    1
2025    1
2026    1
2027    1
Name: res_has_panel, Length: 2028, dtype: int64

In [5]:
def fraud_tagged_reason(x):
    reason = ''
    if x['lt50m']:
        reason = 'lt50m'
    elif x['lt100m'] == 1:
        reason = 'lt100m'
    elif x['gt10km']:
        reason = 'gt10km'
    elif x['gt20km']:
        reason = 'gt20km'
    
    return reason
        

In [6]:
tagged_df = df[((df['lt100m']== 1 ) | (df['lt50m'] == 1) | (df['gt10km']== 1) | (df['gt20km']== 1) ) & (df['res_has_panel']==0) ]

In [7]:
tagged_df.shape

(61, 31)

In [8]:
pay_riders_orders = tagged_df[
    (tagged_df['payment_difference_between_user_restaurant']<0) & (tagged_df['restaurant_payment_type'] == 0)
]

In [9]:
pay_riders_orders.shape

(49, 31)

In [10]:
pay_riders_orders['time_pickedup_at_to_delivered_at'] = pay_riders_orders['time_pickedup_at_to_delivered_at'].apply(lambda x: round(x,2))

<ipython-input-10-2a617fbc5743>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pay_riders_orders['time_pickedup_at_to_delivered_at'] = pay_riders_orders['time_pickedup_at_to_delivered_at'].apply(lambda x: round(x,2))


In [11]:
pay_riders_orders['tagged_reason'] = pay_riders_orders.apply(lambda x: fraud_tagged_reason(x), axis =1 )

<ipython-input-11-e02144fbc3d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pay_riders_orders['tagged_reason'] = pay_riders_orders.apply(lambda x: fraud_tagged_reason(x), axis =1 )


In [12]:
tagged_providers_grouped = pay_riders_orders.groupby(
    'provider_id',
).agg(
    {
        'coupon_code': list,
        'order_id': list,
        'time_pickedup_at_to_delivered_at' : list,
        'count_coordinates_pickedup_delivered' : list,
        'tagged_reason' : list,
    }
    
).reset_index()
tagged_providers_grouped['tagged_order_count'] = tagged_providers_grouped['order_id'].apply(lambda x: len(x))
tagged_providers_grouped.shape

(29, 7)

In [13]:
all_providers = df.groupby(
    'provider_id',
).agg(
    {
        'order_id': list,
    }
    
).reset_index()
all_providers['total_order_count'] = all_providers['order_id'].apply(lambda x: len(x))
all_providers.shape

(385, 3)

In [14]:
all_providers = all_providers.drop(
    columns= [
        'order_id'
    ]
)
merge_providers = tagged_providers_grouped.merge(
    all_providers,
    left_on = 'provider_id',
    right_on= 'provider_id',
    how = 'left'
)
merge_providers['tagged_ratio'] = merge_providers['tagged_order_count'] / merge_providers['total_order_count']
merge_providers.shape

(29, 9)

In [15]:
merge_providers['tagged_order_count'].sum()

49

In [16]:
merge_providers.columns

Index(['provider_id', 'coupon_code', 'order_id',
       'time_pickedup_at_to_delivered_at',
       'count_coordinates_pickedup_delivered', 'tagged_reason',
       'tagged_order_count', 'total_order_count', 'tagged_ratio'],
      dtype='object')

In [17]:
date = datetime.strftime(datetime.now() - timedelta(number), '%Y-%m-%d')
merge_providers['order_date'] = date
merge_providers.shape

(29, 10)

In [18]:
file_creds = "db_creds_analytics.json"
dbmanager = DatabaseManager(file=file_creds)
write_to_db(
    table="table_name",
    db=dbmanager,
    df=merge_providers,
)
dbmanager.close()

[('0a5d3f8a-5dc8-483c-8530-fe5ae17fd420', 'BLAST50', 'JJ-30DCF-10', '132.9', '1015', 'gt10km', '1', '2', '0.5', '2021-08-01'), ('1daefedb-dc2f-4c37-83bf-6b6312724f91', 'BLAST50', 'TI-409E0-1', '11.85', '139', 'lt100m', '1', '10', '0.1', '2021-08-01'), ('2428aff0-5890-4758-aab1-7e73243b43e3', 'BLAST50,BLAST50', 'SA-A5EF5-25,MA-485D9-4', '5.27,1.02', '62,12', 'lt50m,lt50m', '2', '10', '0.2', '2021-08-01'), ('25f3ceac-56a8-4917-91ad-281f62784291', 'BLAST50', 'ME-D9E0C-4', '3.27', '38', 'lt50m', '1', '12', '0.08333333333333333', '2021-08-01'), ('3356b90e-be49-40db-ab38-360783404ef3', 'BLAST50', 'MS-25A96-2', '0.32', '3', 'lt50m', '1', '7', '0.14285714285714285', '2021-08-01')]
